In [1]:
import random
import json
import os
from apache_atlas.utils.Constants import EndRelations, TypeNames
from apache_atlas.utils.API import API, HTTPMethod
from apache_atlas.client.ApacheAtlas import ApacheAtlasClient

atlas_section = ApacheAtlasClient(
    "http://10.100.100.61:21000",
    "admin",
    "admin"
)

### Criação das Tabelas

In [ ]:
# Executado...

groups_sia ={
    #'AB': 'APAC de Cirurgia Bariátrica',
    'ABO': 'APAC de Acompanhamento Pós Cirurgia Bariátrica',
    'ACF': 'APAC de Confecção de Fístula',
    'AD': 'APAC de Laudos Diversos',
    'AM': 'APAC de Medicamentos',
    'AMP': 'APAC de Acompanhamento Multiprofissional',
    'AN': 'APAC de Nefrologia',
    'AQ': 'APAC de Quimioterapia',
    'AR': 'APAC de Radioterapia',
    'ATD': 'APAC de Tratamento Dialítico',
    'BI': 'Boletim de Produção Ambulatorial individualizado',
    'PA': 'Produção Ambulatorial',
    'PS': 'RAAS Psicossocial',
    'SAD': 'RAAS de Atenção Domiciliar'
}

for acronymus, name in groups_sia.items():
    atlas_section.entity.create_entity_dt_table({
       'name': f"{name} ({acronymus})",
       'acronymus': acronymus,
       'description': f'Tabela de {name} da SIA' 
    }, 'SIA')

### Criação de Colunas

In [ ]:
# Executado...

import json
import os

pasta = 'dicionarios'

for arquivo in os.listdir(pasta):
    sigla = arquivo.split('-')[1].split('.')[0]
    caminho_dict = os.path.join(pasta, arquivo)

    atlas_section.entity.create_entity_columns(caminho_dict, sigla)

### Criação de Tabelas Anuais/Mensais

In [ ]:
# Executado...

atlas_section.type.create_type([
   {
      "name": TypeNames.MONTLY_TABLE,
      "description": "Representa uma tabela em um mês específico",
      "superTypes": [TypeNames.ANUAL_TABLE],
      "attributeDefs": [
        {
          "name": "month",
          "typeName": "int",
          "isOptional": True,
          "isUnique": False,
          "isIndexable": False
        }
      ]
    },
])

atlas_section.type.create_type(relationshipDefs=[
    {
    "name": "rsh_table_to_columns_time",
    "relationshipCategory": "AGGREGATION",
    "propagateTags": "NONE",
    "endDef1": {
        "type": TypeNames.TABLE,
        "name": EndRelations.END_TABLE_TO_COLUMNS_TIME[0],
        "cardinality": "SET",
        "isContainer": True,
        "isLegacyAttribute": False
    },
    "endDef2": {
        "type": TypeNames.ANUAL_TABLE,
        "name": EndRelations.END_TABLE_TO_COLUMNS_TIME[1],
        "cardinality": "SINGLE",
        "isContainer": False,
        "isLegacyAttribute": False
    }
}, 
])


In [ ]:
# Executado ...

url = atlas_section.search.SEARCH_BY_TYPENAME.format_path({
    'typeName': TypeNames.ANUAL_TABLE
})

response = atlas_section.request(url)

filters = list(filter(
    lambda x: x['attributes']['name'].startswith("DO"),
    response['entities']
)) 

table = atlas_section.search.search_table_by_acronymus("DO")

for i in filters:
    i['attributes'][EndRelations.END_TABLE_TO_COLUMNS_TIME[1]] = {
        'guid': table['guid']
    }

atlas_section.entity.create_multiple_entities(filters)

In [ ]:
# Executado apenas para o SIA
# Deu problema no AM (AM Altura), AR sei lá oq

import os
import json

pasta = 'data'

for arquivo in os.listdir(pasta):
    if arquivo.endswith('sia.json'):
        caminho_arquivo = os.path.join(pasta, arquivo)

        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            dados_json = json.load(f)
        
        for acronymus, lineage in dados_json.items():
            atlas_section.lineage.create_lineage_table(lineage, acronymus)

In [ ]:
# Executado...

atlas_section.entity.create_database_entity({
    'name': 'Database de Tabelas',
    'description': 'Database apenas para agrupar determinadas tabelas (Ignorar)',
    'acronymus': TypeNames.ACRONYMUS_DATABASE_DTC
}, "DataSUS")

atlas_section.entity.create_entity_dt_table({
    "name": "Tabela de Colunas",
    "description": "Tabela de referência para colunas/arquivos alterados por processos (Ignorar)",
    "acronymus": TypeNames.ACRONYMUS_TABLE_DTC
}, f"{TypeNames.ACRONYMUS_DATABASE_DTC}")

### Criação de DataSET Processing

In [ ]:
atlas_section.type.create_type(types=[
    {
      "name": TypeNames.DATASET_PROCESSING_LINEAGE,
      "description": "Representa um cojunto de arquivos de tabelas que passará pro um sequência de processos",
      "superTypes": ["DataSet"],
      "attributeDefs": [
        {
          "name": "files_interval",
          "typeName": f"array<{TypeNames.TABLE_FILE}>",
          "isOptional": True,
          "isUnique": False,
          "isIndexable": False
        },
        {
          "name": "columns",
          "typeName": f"array<{TypeNames.TABLE_COLUMN}>",
          "isOptional": True,
          "isUnique": False,
          "isIndexable": False   
        },
        {
          "name": "id",
          "typeName": f"string",
          "isOptional": True,
          "isUnique": True,
          "isIndexable": False
        },
        {
            "name": "start_time",
            "typeName": "date",
            "isOptional": False,        
            "valuesMinCount": 1,            
            "isIndexable": True,
            "description": "Start time of the ETL process interval."
          },
          {
            "name": "end_time",
            "typeName": "date",
            "isOptional": False,        
            "valuesMinCount": 1,            
            "isIndexable": True,
            "description": "End time of the ETL process interval."
          },
          {
            "name": "output_location",
            "typeName": "string",
            "isOptional": True,                    
            "isUnique": False,
            "isIndexable": False,
            "description": "Output location URI or path for the processed data."
          },
          {
            "name": "records_processed",
            "typeName": "long",
            "isOptional": True,                    
            "isUnique": False,
            "isIndexable": False,
            "description": "Total number of records processed in the ETL batch."
          },
          {
            "name": "executionTime",
            "typeName": "long",
            "isOptional": True,                    
            "isUnique": False,
            "isIndexable": False,
            "description": "Total execution time of the ETL process in milliseconds."
          },
      ]
    }
])

atlas_section.type.create_type(types=[
    {
      "name": TypeNames.DATASET_PROCESSING_LINEAGE_RESULT,
      "description": "Representa um resultado após um processo",
      "superTypes": ["DataSet"],
      "attributeDefs": [
        {
          "name": "files_interval",
          "typeName": f"array<{TypeNames.ANUAL_TABLE}>",
          "isOptional": True,
          "isUnique": False,
          "isIndexable": False
        },
        {
          "name": "columns",
          "typeName": f"array<{TypeNames.TABLE_COLUMN}>",
          "isOptional": True,
          "isUnique": False,
          "isIndexable": False   
        }
      ]
    }
])

In [ ]:
process_id = 'DO-32132322'

atlas_section.lineage.create_entity_lineage_by_interval_time_anual({
    'start_year': 2017,
    'end_year': 2022,
}, "DO", process_id)

In [ ]:
atlas_section.process.create_process_drop_column_dataset(
    process_id,
    [
        "ORIGEM",
        "ESC2010",
        "SERIESCFAL",
        "OCUP",
        "LINHAA",
        "LINHAB",
        "LINHAC",
        "LINHAD",
        "LINHAII",
        "DTRECORIGA",
        "ESCMAEAGR1",
        "ESCFALAGR1",
        "STDOEPIDEM",
        "STDONOVA",
        "DIFDATA",
        "CONTADOR",
        "DTRECEBIM",
        "VERSAOSIST",
        "CAUSABAS_O",
        "ATESTANTE",
        "STCODIFICA",
        "CODIFICADO",
        "VERSAOSCB",
        "NUMEROLOTE",
        "TPPOS",
        "CODMUNRES",
        "CODMUNNATU",
        "CODMUNOCOR",
        "ATESTADO"
    ]
)

In [ ]:
changed_columns = [
    {
        "name": "SEXO",
        "attributes_to_change": { 
            'domain': ', '.join(["Ignorado", "Homem", "Mulher"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "CAUSABAS",
        "attributes_to_change": { 
            'domain': "CID10 Descrição",
            'type': 'Char (255)'
        }
    },
    {
        "name": "TIPOBITO",
        "attributes_to_change": { 
            'domain': ', '.join(["Fetal", "Não Fetal"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "RACACOR",
        "attributes_to_change": { 
            'domain': ', '.join(["Branca", "Preta", "Amarela", "Parda", "Indígena"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "ESTCIV",
        "attributes_to_change": { 
            'domain': ', '.join(["Solteito", "Casado", "Viúvo", "Separado Judicialmente", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "ESC",
        "attributes_to_change": { 
            'domain': ', '.join(["Nenhuma", "1 a 3 anos", "4 a 7 anos", "8 a 11 anos", "12 a mais", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "LOCOCOR",
        "attributes_to_change": { 
            'domain': ', '.join(["Hospital", "Outro estado saúde", "Domicílio", "Via Pública", "Outros", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "GRAVIDEZ",
        "attributes_to_change": { 
            'domain': ', '.join(["Única", "Dupla", "Tripla e mais", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "SEMAGESTAC",
        "attributes_to_change": { 
            'domain': ', '.join(["Menos de 22 semanas", "22 a 27 semanas", "28 a 31 semanas", "32 a 36 semanas", "37 a 41 semanas", "42 semanas e mais", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "GESTACAO",
        "attributes_to_change": { 
            'domain': ', '.join(["Vaginal", "Cesáreo", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "PARTO",
        "attributes_to_change": { 
            'domain': ', '.join(["Antes" , "Durante", "Depois", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "OBITOPARTO",
        "attributes_to_change": { 
            'domain': ', '.join(["Antes" , "Durante", "Depois", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "OBITOGRAV",
        "attributes_to_change": { 
            'domain': ', '.join(["Sim" , "Não", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "OBITOPUERP",
        "attributes_to_change": { 
            'domain': ', '.join(["Sim até 42 dias" , "Sim, de 43 dias a 1 ano", "Não", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "ASSISTMED",
        "attributes_to_change": { 
            'domain': ', '.join(["Sim" , "Não", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "EXAME",
        "attributes_to_change": { 
            'domain': ', '.join(["Sim" , "Não", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "CIRURGIA",
        "attributes_to_change": { 
            'domain': ', '.join(["Sim" , "Não", "Ignorado"]),
            'type': 'Char (255)'
        }
    },
    {
        "name": "NECROPSIA",
        "attributes_to_change": { 
            'domain': ', '.join(["Sim" , "Não", "Ignorado"]),
            'type': 'Char (255)'
        }
    }
]

atlas_section.process.create_process_alter_column_dataset(
    changed_columns,
    process_id,
    'DO', {
        'name': 'Grupo Transformação',
        'description': 'Replace de valores númericos por sua respectiva descrição'
    }
)

In [ ]:
columns_renamed = {
    "DTOBITO": "Data do Óbito",
    "HORAOBITO": "Hora do Óbito",
    "DTNASC": "Data de Nascimento",
    "TIPOBITO": "Tipo de Óbito",
    "NATURAL": "Naturalidade",
    "IDADE": "Idade",
    "SEXO": "Sexo",
    "RACACOR": "Raça/Cor",
    "ESTCIV": "Estado Civil",
    "ESC": "Escolaridade",
    "LOCOCOR": "Local da Ocorrência",
    "CODESTAB": "Hospital",
    "ASSISTMED": "Assintência Médica",
    "NECROPSIA": "Necrópsia",
    "CAUSABAS": "Causa Básica",
    #"CAUSABAS_keep": "Categoria CID",
    "DTATESTADO": "Data do Atestado",
    "DTCADASTRO": "Data do Cadastro",
    "COMUNSVOIM": "Cidade do Paciente"
}

data = [
    { 
        'name': old_name,
        'attributes_to_change': {
            'name': new_name
        } 
    } for old_name, new_name in columns_renamed.items()
]

atlas_section.process.create_process_alter_column_dataset(
    data,
    process_id,
    'DO'
)